# Simulation of the N-Body Problem
## Created by Jack Le - June 20th, 2021
### Sources used:
* https://en.wikipedia.org/wiki/N-body_problem
* https://en.wikipedia.org/wiki/N-body_simulation
* https://en.wikipedia.org/wiki/Leapfrog_integration
* https://medium.com/swlh/creating-3d-video-visualization-with-matplotlib-python-data-visualization-series-d8f5dfe1c460

In [1]:
import numpy as np
np.random.seed(0) # seeds the generator so randoms will stay same across each run
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from tqdm.notebook import tqdm
from mpl_toolkits import mplot3d
from PIL import Image
import os.path
from multiprocessing import Process
from time import sleep

### Force Equations
[//]: # "F_{ij} = G\frac{m_im_j}{\left \| r_j-r_i \right \|^2}*\frac{(r_j-r_i)}{\left \| r_j-r_i \right \|}=G\frac{m_im_j(r_j-r_i)}{\left \| r_j-r_i \right \|^3}"
<img src="images/force.png"> 

### Acceleration Equation
[//]: # "a_i = G \sum_{j\neq i}m_j\frac{r_j-r_i}{\left \| r_j-r_i \right \|^3}"
<img src="images/accel.png">

### Leapfrog Implementation
<img src="images/leapfrog.svg">

In [2]:
class N_body:
    
    def __init__(self, timestep=0.01, softening=0.1, length=10, N=100, total_mass=20.0, dirname='other'):
        """
        Initializes the class and the N-body simulation
        :float G: coefficient of Newton's Gravitational Constant, 6.67 × 10-11 Newtons kg-2 m2
        :int N: the number of bodies in the simulation
        :float t: the current time in the simulation
        :float length: the length (time) of the simulation
        :float t_end: the time when simulation ends (t + length)
        :float timestep: amount of time passed per step
        :float softening: the softening parameter used for the softening trick
        :float total_mass: the total mass of all particles in the system
        """
        self.G = 1.0
        self.N = N
        self.t = 0
        self.length = length
        self.t_end = self.t + self.length
        self.timestep = timestep
        self.softening = softening
        self.total_mass = total_mass
        self.dirname=dirname
    
    
    def simulate(self):
        """
        Main simulation function
        :np.array positions: N x 3 matrix storing positions in the x, y, and z dimensions
        :np.array masses: N x 1 vector storing the masses of each particle
        :np.array velocities: N x 3 matrix storing velocities in the x, y, and z dimensions
        :np.array accelerations: N x 3 matrix storing accelerations in the x, y, and z dimensions
        :int num_timesteps: the number of timesteps between initial t and t_end
        :np.array pos_s: storage array of positions, used to store previous positions for plotting
        """
        positions = np.random.randn(self.N, 3)
        masses = self.total_mass * np.ones((self.N, 1))/self.N
        velocities = np.random.randn(self.N, 3)
        velocities -= np.mean(masses * velocities, axis=0) / np.mean(masses)
        accelerations = self.get_accelerations(positions, masses)
        kinetic_energies, potential_energies = self.get_energies(positions, velocities, masses)
        
        num_timesteps = int(np.ceil(self.t_end/self.timestep))
        
        # saves the previous positions for plotting
        self.pos_s = np.zeros((self.N,3,num_timesteps+1))
        self.pos_s[:,:,0] = positions
        
        # save initial energies for plotting
        self.KE_s = np.zeros(num_timesteps+1)
        self.KE_s[0] = kinetic_energies
        self.PE_s = np.zeros(num_timesteps+1)
        self.PE_s[0] = potential_energies
        self.t_all = np.arange(num_timesteps+1)
        
        # uses leapfrog kick-drift-kick to simulate time
        # 1. it first has a half step kick in velocity, vf = vi + t/2 * a
        # 2. then it has a drift, rf = ri + t * vf
        # 3. it then updates the acceleartions
        # 4. then it has another kick, vf = vi + t/2 * a
        angle = 70
        for i in tqdm(range(num_timesteps)):
            # first kick
            velocities += (self.timestep/2.0) * accelerations
            
            # drift
            positions += velocities * self.timestep
            
            # update accelerations
            accelerations = self.get_accelerations(positions, masses)
            
            # second kick
            velocities += (self.timestep/2.0) * accelerations
            
            # move to next timestep
            self.t += self.timestep
            
            # update energies of the system
            kinetic_energies, potential_energies = self.get_energies(positions, velocities, masses)
            
            # save positions for plotting trail
            self.pos_s[:,:,i+1] = positions
            self.KE_s[i+1] = kinetic_energies
            self.PE_s[i+1] = potential_energies

            # graphs the simulation
            fig = plt.figure(figsize=(16,10), dpi=80, facecolor='white')
            gs = gridspec.GridSpec(nrows=1, ncols=3)
            ax1 = fig.add_subplot(gs[0, 0:2], projection="3d")
            ax1.set(xlim=(-2, 2), ylim=(-2, 2), zlim=(-2,2))
            #ax1._axis3don = False
            ax2 = fig.add_subplot(gs[0, 2])

            angle += 0.25
            angle %= 360

            x_trail = self.pos_s[:, 0, max(0, i-10):i+1]
            y_trail = self.pos_s[:, 1, max(0, i-10):i+1]
            z_trail = self.pos_s[:, 2, max(0, i-10):i+1]
            ax1.set_title(f'N-Body Simulation with {self.N} bodies: time {i+1}/{num_timesteps}')
            ax1.scatter3D(x_trail, y_trail, z_trail, s=10, color=[.7,.7,1])
            ax1.scatter3D(positions[:,0], positions[:,1], positions[:,2], s=10, color='blue')
            ax1.view_init(30,angle)
            
            ax2.scatter(y=self.t_all[:i+1], x=self.KE_s[:i+1], color='red', s=1, label='KE')
            ax2.scatter(y=self.t_all[:i+1], x=self.PE_s[:i+1], color='blue', s=1, label='PE')
            ax2.scatter(y=self.t_all[:i+1], x=self.KE_s[:i+1]+self.PE_s[:i+1], color='black', s=1, label='TE')
            ax2.legend(loc="upper right")
            ax2.set(ylim=(0, num_timesteps), xlim=(-300, 300))
            ax2.xaxis.tick_top()
            ax2.invert_yaxis()
            ax2.set_xlabel("Energy")
            ax2.set_ylabel("Timestep")
            ax2.xaxis.set_label_position('top')
            ax2.yaxis.set_label_position('right') 

            #Save figure
            plt.savefig(f'frame_images/{self.dirname}/3d_nbody_{i}.png',dpi=100)    
            fig.clf()
            plt.close(fig)
            
            #plt.show()
            
            
            
    def get_accelerations(self, positions, masses):
        """
        Calculates the accelerations of each particle given its mass and position, based on Newton's Laws of Gravitation
        :param positions: N x 3 matrix storing positions in the x, y, and z dimensions
        :param masses: N x 1 vector storing the masses of each particle
        :return accelerations: N x 3 matrix storing accelerations in the x, y, and z dimensions
        """
        
        accelerations = None # will store accelerations in each dimension
        
        pos_x = positions[:, 0:1] # N x 1 vector of x positions
        pos_y = positions[:, 1:2] # N x 1 vector of y positions
        pos_z = positions[:, 2:3] # N x 1 vector of z positions
        
        sep_x = pos_x.T - pos_x # stores the separation r_j - r_i in x dimension
        sep_y = pos_y.T - pos_y # stores the separation r_j - r_i in y dimension
        sep_z = pos_z.T - pos_z # stores the separation r_j - r_i in z dimension
        
        # stores the denominator in acceleration equation
        denom = sep_x**2 + sep_y**2 + sep_z**2 + self.softening**2
        denom[denom>0] **= -3/2
        
        a_x = self.G * (sep_x * denom) @ masses
        a_y = self.G * (sep_y * denom) @ masses
        a_z = self.G * (sep_z * denom) @ masses
        
        accelerations = np.hstack((a_x, a_y, a_z))
        
        return accelerations
    
    
    def get_energies(self, positions, velocities, masses):
        """
        Calculates the total kinetic and potential energies of the system
        :param positions: N x 3 matrix storing positions in the x, y, and z dimensions
        :param velocities: N x 3 matrix storing velocities in the x, y, and z dimensions
        :param masses: N x 1 vector storing the masses of each particle
        """
        kinetic_energies = (1.0/2.0) * np.sum(np.sum(masses * velocities**2)) # total kinetic energy of the system
        
        pos_x = positions[:, 0:1] # N x 1 vector of x positions
        pos_y = positions[:, 1:2] # N x 1 vector of y positions
        pos_z = positions[:, 2:3] # N x 1 vector of z positions
    
        sep_x = pos_x.T - pos_x # stores the separation r_j - r_i in x dimension
        sep_y = pos_y.T - pos_y # stores the separation r_j - r_i in y dimension
        sep_z = pos_z.T - pos_z # stores the separation r_j - r_i in z dimension
        
        # stores the denominator in acceleration equation
        denom = np.sqrt(sep_x**2 + sep_y**2 + sep_z**2)
        denom[denom>0] = 1/denom[denom>0]
        
        potential_energies = self.G * np.sum(np.sum(np.triu(-(masses*masses.T)*denom,1))) # total potential energy of the system
        
        return kinetic_energies, potential_energies

In [3]:
def save(N, length, dirname):
    # Gathers the images
    png_count = 5000
    files = []
    for i in range(png_count):
        seq = str(i)
        fname = f'frame_images/{dirname}/3d_nbody_'+ seq +'.png'
        if os.path.isfile(fname):
            files.append(fname)
            
    # Create the frames
    frames = []
    for i in files:
        new_frame = Image.open(i)
        frames.append(new_frame)

    # Save into a GIF file that loops forever   
    frames[0].save(f'simulations/{N}_body_simulation_length_{length}.gif', format='GIF',
                   append_images=frames[1:],
                   save_all=True,
                   duration=30, loop=0)
    
    for frame in frames:
        frame.close()

In [4]:
def create_sim(N=100, length=10):
    timestep=0.01
    softening=0.1
    total_mass=20.0
    dir_name = f'sim_{N}_body_{length}_len'
    dir2_name = './frame_images/' + dir_name
    !rm -r {dir2_name}
    !mkdir {dir2_name}
    print('Creating simulation with parameters:\n' +
          f'Timestep length = {timestep}\n' +
          f'Softening parameter = {softening}\n' +
          f'Simulation length = {length}\n' +
          f'Num of bodies = {N}\n' +
          f'Total mass = {total_mass}')
    simulation = N_body(timestep=timestep, softening=softening, length=length, N=N, total_mass=total_mass, dirname=dir_name)
    simulation.simulate()
    save(N=N, length=length, dirname=dir_name)

In [5]:
# sets the file limit so that it can do all this multiprocessing
import resource
soft, hard = resource.getrlimit(resource.RLIMIT_NOFILE)
resource.setrlimit(resource.RLIMIT_NOFILE, (hard, hard))

In [6]:
# create_sim(N=2, length=20)
p1 = Process(target=create_sim, args=[2, 20])
# create_sim(N=5, length=20)
p2 = Process(target=create_sim, args=[5, 20])

# starting processes
p1.start()
sleep(1)
p2.start()
sleep(1)

# wait until processes are finished
p1.join()
p2.join()

# all processes finished
print("Done!")

Creating simulation with parameters:
Timestep length = 0.01
Softening parameter = 0.1
Simulation length = 20
Num of bodies = 2
Total mass = 20.0


  0%|          | 0/2000 [00:00<?, ?it/s]

Creating simulation with parameters:
Timestep length = 0.01
Softening parameter = 0.1
Simulation length = 20
Num of bodies = 5
Total mass = 20.0


  0%|          | 0/2000 [00:00<?, ?it/s]

Done!


In [7]:
# create_sim(N=10, length=20)
p3 = Process(target=create_sim, args=[10, 20])
# create_sim(N=30, length=20)
p4 = Process(target=create_sim, args=[30, 20])

# starting processes
p3.start()
sleep(1)
p4.start()
sleep(1)

# wait until processes are finished
p3.join()
p4.join()

# all processes finished
print("Done!")

Creating simulation with parameters:
Timestep length = 0.01
Softening parameter = 0.1
Simulation length = 20
Num of bodies = 10
Total mass = 20.0


  0%|          | 0/2000 [00:00<?, ?it/s]

rm: cannot remove './frame_images/sim_30_body_20_len': No such file or directory
Creating simulation with parameters:
Timestep length = 0.01
Softening parameter = 0.1
Simulation length = 20
Num of bodies = 30
Total mass = 20.0


  0%|          | 0/2000 [00:00<?, ?it/s]

Done!


In [8]:
# create_sim(N=50, length=20)
p5 = Process(target=create_sim, args=[50, 20])
# create_sim(N=100, length=20)
p6 = Process(target=create_sim, args=[100, 20])

# starting processes
p5.start()
sleep(1)
p6.start()
sleep(1)

# wait until processes are finished
p5.join()
p6.join()

# all processes finished
print("Done!")

rm: cannot remove './frame_images/sim_50_body_20_len': No such file or directory
Creating simulation with parameters:
Timestep length = 0.01
Softening parameter = 0.1
Simulation length = 20
Num of bodies = 50
Total mass = 20.0


  0%|          | 0/2000 [00:00<?, ?it/s]

rm: cannot remove './frame_images/sim_100_body_20_len': No such file or directory
Creating simulation with parameters:
Timestep length = 0.01
Softening parameter = 0.1
Simulation length = 20
Num of bodies = 100
Total mass = 20.0


  0%|          | 0/2000 [00:00<?, ?it/s]

Done!
